# Solution idea is from
#### Original (but not a notebook) remark: not Python 3 used
https://pclubiiti.github.io/blog/Inroduction-toSAT-solving/

Uses http://binarypuzzle.com


# completed by Peter Gragert 
for any quadratic BP of size 2n x 2n

# a Binary Puzzel is a 2n x 2n quadratic scheme
consisting of 0s and 1s and .s (denoting not yet known) boxes.


# The rules

Each binary puzzle should be solved according to the following rules:

- Each box should contains a zero or a one.
- No more than two similar numbers next to or below each other are allowed.
- Each row and each column should contain an equal number of zeros and ones.
- Each row is unique and each column is unique. 

<p style="font-family:Courier; color:Blue; font-size: 20px;">
A really good bindary puzzel should have ONE solution and can be solved without gessing.</p>

In [19]:
# example of a 10 x 10 puzzel (ONE solution)

game = """
...00....1
........01
..0...1.0.
.1...1....
.0.0.11...
1.....1...
.0...1....
...1.1.11.
..........
..00......
"""

from z3 import *
import numpy as np # to show (first) solution matrice
#solutions will be saved in a dictionary of np.matrices
s = Solver()
# PKHG>for DBG vals a variable to access (later)
vals = []

sol = None #PKHG>for DBG too

def SolvePuzzel(game, maxCounter = 2, show_sol = False):
    """
    before use ececute: s = Solver() 
    game is to be solved
    default counter maximized by 2 to see if
    there is more than ONE solution
    solutions as np.matrices are stored in a dictionary and returned
    """
    global vals
    
    #for controlling the number of solutions!
    result = dict()
    counter = 0 #while loop uses it
    
    #check input game!
    try:
        game = [y for y in filter(lambda x:x in '01.', game)]
        sizeXsize = len(game)
        size = int(math.sqrt(sizeXsize))
        assert(size * size == sizeXsize) 
        half_int_size = int(size/2)
        assert(size == 2 * int(size/2)) #even number of rows and columns
    except:
        return("check input game")
    
    #  a list cordinates to access (later) rows or columns)
    vs = [(r,c) for r in range(size) for c in range(size)]

    #  all cells declared as BitVectors Z3-variables
    # _ used to   rows and colum indices
    # half_int_size should suffice in BitVec ...
    bV  = [[z3.BitVec('v_%d_%d' % (r,c), half_int_size)  for c in range(size)] 
      for r in range(size)]
    
    #initializing all variables using the given info
    # either 0 or 1 or condition BINARY (value 0 or 1)
    
    # the z3-solver global for DBG
    #s = Solver()

    for ((r,c), val) in zip(vs, game):
        if val == '.':  s.add(0 <= bV[r][c], bV[r][c] <= 1)
        else:           s.add(bV[r][c] == (int(val)))

    #helpers for row and column indices 
    def row(n):   return [(n, i) for i in range(size)]
    def col(n):   return [(i, n) for i in range(size)]

    #Puzzel condition: rows, columns must sum up to size/2
    for i in range(size):
        s.add(z3.Sum([bV[r][c] for r,c in row(i)]) == (half_int_size))
        s.add(z3.Sum([bV[r][c] for r,c in col(i)]) == (half_int_size))

    # puzzel condition at moost two 1s or 0s in row resp. column
    # help functions for 3 consecutive cells (rows or cols)
    def row3(n, m):   return [(n, i) for i in range(m, m + 3)]
    def col3(n, m):   return [(i, n) for i in range(m, m + 3)]
    
    for i in range(size):
        for j in range(size-2):
            #at most two 1s
            s.add(z3.Sum([bV[r][c] for r,c in row3(i,j)]) <= 2)
            #at most two 0s
            s.add(z3.Sum([bV[r][c] for r,c in row3(i,j)]) >= 1)
            # same for the columns
            s.add(z3.Sum([bV[r][c] for r,c in col3(i,j)]) <= 2)
            s.add(z3.Sum([bV[r][c] for r,c in col3(i,j)]) >= 1)

    
    #That just leaves Rule #4 Each row is unique 
    # and each column is unique. Z3 has a Unique keyword.
    s.add(z3.Distinct([z3.Concat([bV[r][c] for r,c in row(i)])
                       for i in range(size)]))
    s.add(z3.Distinct([z3.Concat([bV[r][c] for r,c in col(i)]) 
                       for i in range(size)]))
    # learned from Nicolaj ;-)
    while s.check() == sat and counter < maxCounter:
        m = s.model()
        #PKHG>OK possible too:
        #sol = [[m[bV[r][c]].as_string() for c in range(size)] for r in range(size)]
       
        sol = [[m[bV[r][c]].as_long() for c in range(size)] for r in range(size)]
        #PKHG>DBG print("type of (0,0) element = " , type(m[bV[0][0]]))
        
        result[counter] = np.matrix(sol) #zero based dict!
        counter += 1
        
        if show_sol:
            print_matrix(sol)
        
        # deny found solution, search for second or ... one
        vals = [bV[i][j] == m.evaluate(bV[i][j]) for j in range(size)
            for i in range(size)]
        s.add(Not(And(vals)))
        #repeat ;-)
        
    if counter == 1:
        print("Exactly ONE solution")
    elif counter > 1:
        print("several solutions found:" , counter)
    else:
        print("no solution")
    return result



In [20]:
# example of a 10 x 10 puzzel (ONE solution)

game = """
...00....1
........01
..0...1.0.
.1...1....
.0.0.11...
1.....1...
.0...1....
...1.1.11.
..........
..00......
"""

from z3 import *
import numpy as np # to show (first) solution matrice
s = Solver()
vals = []
sol = None

def SolvePuzzel(game, maxCounter = 2, show_sol = False):
    """
    game is to be solved
    default counter maximized by 2 to see if
    there is more than ONE solution
    solutions are stored in a dictionary and returned
    
    PKHG>HOW????
    
    """
    global vals
    #for controlling the number of solutions!
    result = dict()
    counter = 0 #while loop uses it
    
    #check input game!
    try:
        game = [y for y in filter(lambda x:x in '01.', game)]
        sizeXsize = len(game)
        size = int(math.sqrt(sizeXsize))
        assert(size * size == sizeXsize) 
        half_int_size = int(size/2)
        assert(size == 2 * int(size/2)) #even number of rows and columns
    except:
        return("check input game")
    #  a list cordinates to access (later) rows or columns)
    vs = [(r,c) for r in range(size) for c in range(size)]

    #  all cells declared as BitVectors Z3-variables
    # _ used to   rows and colum indices
    bV  = [[z3.BitVec('v_%d_%d' % (r,c), 5)  for c in range(size)] 
      for r in range(size)]
    
    #initializing all variables using the given info
    # either 0 or 1 or condition BINARY (value 0 or 1)
    
    # the z3-solver
    #s = Solver()

    for ((r,c), val) in zip(vs, game):
        if val == '.':  s.add(0 <= bV[r][c], bV[r][c] <= 1)
        else:           s.add(bV[r][c] == (int(val)))

    #helpers for row and colum indices 
    def row(n):   return [(n, i) for i in range(size)]
    def col(n):   return [(i, n) for i in range(size)]

    #Puzzel condition: rows, columns must sum up to size/2
    for i in range(size):
        s.add(z3.Sum([bV[r][c] for r,c in row(i)]) == (size/2))
        s.add(z3.Sum([bV[r][c] for r,c in col(i)]) == (size/2))

    # puzzel condition at moost two 1s or 0s in row resp. column
    # help functions for 3 consecutive cells (rows or cols)
    def row3(n,m):   return [(n, i) for i in range(m,m+3)]
    def col3(n,m):   return [(i, n) for i in range(m,m+3)]
    
    for i in range(size):
        for j in range(size-2):
            #at most two 1s
            s.add(z3.Sum([bV[r][c] for r,c in row3(i,j)]) <= 2)
            #at most two 0s
            s.add(z3.Sum([bV[r][c] for r,c in row3(i,j)]) >= 1)
            # same for the columns
            s.add(z3.Sum([bV[r][c] for r,c in col3(i,j)]) <= 2)
            s.add(z3.Sum([bV[r][c] for r,c in col3(i,j)]) >= 1)

    
    #That just leaves Rule #4 Each row is unique 
    # and each column is unique. Z3 has a Unique keyword.
    s.add(z3.Distinct([z3.Concat([bV[r][c] for r,c in row(i)])
                       for i in range(size)]))
    s.add(z3.Distinct([z3.Concat([bV[r][c] for r,c in col(i)]) 
                       for i in range(size)]))

    #result = s.check()
    while s.check() == sat and counter < maxCounter:
        m = s.model()
        #PKHG>OK sol = [[m[bV[r][c]].as_string() for c in range(size)] for r in range(size)]
        sol = [[m[bV[r][c]].as_long() for c in range(size)] for r in range(size)]
        #PKHG>DBG print("type of (0,0) element = " , type(m[bV[0][0]]))
        result[counter] = np.matrix(sol) #zero based dict!
        counter += 1
        if show_sol:
            print_matrix(sol)
        if counter >= 1:
        #PKHG>want to save first resp. all solutions
            'todo'
        # deny found solution
        vals = [bV[i][j] == m.evaluate(bV[i][j]) for j in range(size)
            for i in range(size)]
        #PKHG>????
        '''
        for i in range(2):
            print(i, vals[i])
        '''
        s.add(Not(And(vals)))
    if counter == 1:
        print("Exactly ONE solution")
    elif counter > 1:
        print("several solutions found:" , counter)
    else:
        print("no solution")
    return result


# example from original (see link above)

In [21]:
s = Solver()
game = """
...00....1
........01
..0...1.0.
.1...1....
.0.0.11...
1.....1...
.0...1....
...1.1.11.
..........
..00......
"""
SolvePuzzel(game, 3, True)

[[0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
 [1, 0, 1, 1, 0, 0, 1, 0, 0, 1],
 [1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1, 1, 0],
 [0, 0, 1, 0, 0, 1, 1, 0, 1, 1],
 [1, 1, 0, 1, 0, 0, 1, 0, 0, 1],
 [1, 0, 1, 0, 1, 1, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 0, 1, 0, 0, 1],
 [1, 1, 0, 0, 1, 0, 0, 1, 1, 0]]
Exactly ONE solution


{0: matrix([[0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
         [1, 0, 1, 1, 0, 0, 1, 0, 0, 1],
         [1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
         [0, 1, 0, 0, 1, 1, 0, 1, 1, 0],
         [0, 0, 1, 0, 0, 1, 1, 0, 1, 1],
         [1, 1, 0, 1, 0, 0, 1, 0, 0, 1],
         [1, 0, 1, 0, 1, 1, 0, 1, 0, 0],
         [0, 0, 1, 1, 0, 1, 0, 1, 1, 0],
         [0, 1, 0, 1, 1, 0, 1, 0, 0, 1],
         [1, 1, 0, 0, 1, 0, 0, 1, 1, 0]])}

# Check 

In [22]:
s = Solver()
game_changed = """
...0......
........0.
..0...1.0.
.1...1....
.0.0.11...
1.....1...
.0...1....
...1.1.11.
..........
..00......
"""
results = SolvePuzzel(game_changed, 1000)
print("first result:")
results[0]

several solutions found: 109
first result:


matrix([[0, 1, 1, 0, 1, 1, 0, 0, 1, 0],
        [0, 0, 1, 1, 0, 0, 1, 1, 0, 1],
        [1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        [0, 1, 0, 0, 1, 1, 0, 0, 1, 1],
        [1, 0, 1, 0, 0, 1, 1, 0, 0, 1],
        [1, 1, 0, 1, 0, 0, 1, 1, 0, 0],
        [0, 0, 1, 0, 1, 1, 0, 0, 1, 1],
        [0, 1, 0, 1, 0, 1, 0, 1, 1, 0],
        [1, 0, 1, 1, 0, 0, 1, 0, 0, 1],
        [1, 1, 0, 0, 1, 0, 0, 1, 1, 0]])

# hard puzzel 12 x 12 (from link above)

In [23]:
s = Solver()
P_hard_12x12_1 = """
.......00.1.
..1.......1.
...0.0..0..0
.....0......
.11.1..1.0..
.11.0.00....
..........1.
1..00...0...
............
..1.....1...
...0..0..0.0
..10...0....
"""
opls = SolvePuzzel(P_hard_12x12_1)

Exactly ONE solution


In [24]:
opls[0]

matrix([[1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0],
        [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
        [1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0],
        [1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1],
        [0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0],
        [0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1],
        [1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0],
        [1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1],
        [0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1],
        [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1]])

# some smaller BPs

In [25]:
s = Solver()
g2 = """
1.
..
"""
SolvePuzzel(g2)

Exactly ONE solution


{0: matrix([[1, 0],
         [0, 1]])}

In [26]:
s = Solver()
g4 = """
0...
....
1...
..10
"""
g4example = SolvePuzzel(g4,100)

several solutions found: 8


# saving for later

In [27]:
import shelve

In [28]:
#help(shelve)

In [29]:
g4example.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7])

In [30]:
resultdb = shelve.open("puzzelg4")
resultdb['g4'] = g4example
resultdb.close()


#  retrieve all 8 solutions 

In [31]:
resultdb = shelve.open("puzzelg4")
[el for el in resultdb.keys()]

['g4']

In [32]:
#resultdb['g4'] = object
for i in range(7):
    print("oplossing " + str(i) + "\n", resultdb['g4'][i] )

oplossing 0
 [[0 0 1 1]
 [0 1 0 1]
 [1 1 0 0]
 [1 0 1 0]]
oplossing 1
 [[0 1 0 1]
 [0 1 1 0]
 [1 0 0 1]
 [1 0 1 0]]
oplossing 2
 [[0 1 1 0]
 [0 1 0 1]
 [1 0 0 1]
 [1 0 1 0]]
oplossing 3
 [[0 1 0 1]
 [1 0 1 0]
 [1 0 0 1]
 [0 1 1 0]]
oplossing 4
 [[0 1 0 1]
 [1 0 0 1]
 [1 0 1 0]
 [0 1 1 0]]
oplossing 5
 [[0 1 0 1]
 [0 0 1 1]
 [1 1 0 0]
 [1 0 1 0]]
oplossing 6
 [[0 0 1 1]
 [1 0 0 1]
 [1 1 0 0]
 [0 1 1 0]]


# shelves should be closed after use

In [33]:
resultdb.close()

In [34]:
!ls *.db

puzzelg4.db


In [35]:
mydb = shelve.open("puzzelg4")
res = mydb['g4']
mydb.close()

# (easy one) check if different

In [36]:
res[0] == res[1]

matrix([[ True, False, False,  True],
        [ True,  True, False, False],
        [ True, False,  True, False],
        [ True,  True,  True,  True]])